In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
import torch
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from gensim.models import word2vec
import pandas as pd
import numpy as np 
import random
random.seed(2022)
np.random.seed(2022)

In [ ]:
!pip install --upgrade gensim
!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-1.10.0+cu113.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-1.10.0+cu113.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

In [ ]:
df_train=pd.read_csv("criteo_sampled/criteo_train.csv",index_col=0)
df_test=pd.read_csv("criteo_sampled/criteo_test.csv",index_col=0)

In [ ]:
# load causal weighting
ate_list=[]
ate=pd.read_excel("feats_ate_x13.xlsx",index_col=0)
for i in [c for c in df_train.columns[:-4]]+["visit","treatment"]:
    ate_list.append(float(ate[ate['Feature']==i]["ATE"].values))
ate_list

[-0.01388679662724193,
 0.05071538371534517,
 0.001328853128405821,
 -0.009331114629769072,
 0.1933108932875426,
 0.0008912893595676433,
 -0.0002958249515911775,
 -0.009201954095841196,
 0.1034574925533489,
 0.0741312293943781,
 -0.01664079180275199,
 0.1331562899243786,
 0.9310119697738223,
 0.007447354046319239]

In [ ]:
# load edge index
edge_index=pd.read_csv('edge_index_criteo.csv')
edge_index=torch.from_numpy(np.transpose(np.array(edge_index)))

In [ ]:
# load node embedding
model_dw=word2vec.Word2Vec.load("deepwalk_10d_x13.model")
model_n2v=word2vec.Word2Vec.load("Node2Vec_10d_x13.model")

lst_dw=[]
lst_n2v=[]
for i in range(14):
    lst_dw.append(model_dw.wv[i])
    lst_n2v.append(model_n2v.wv[i])
len(lst_n2v)

14

In [ ]:
from scipy.stats import entropy
def R2(y_predicted, y_actual):
    y_predicted = np.asarray(y_predicted, dtype=float)
    y_actual = np.asarray(y_actual, dtype=float)

    R2 = 1 - np.sum(np.square(y_actual-y_predicted)) / np.sum(np.square(y_actual-np.mean(y_actual)))
    return R2

def MAE(y_predicted, y_actual):
    y_predicted = np.asarray(y_predicted, dtype=float)
    y_actual = np.asarray(y_actual, dtype=float)

    MAE = np.mean(abs(y_actual-y_predicted))
    return MAE

def RMSE(y_predicted, y_actual):
    y_predicted = np.asarray(y_predicted, dtype=float)
    y_actual = np.asarray(y_actual, dtype=float)

    RMSE = np.sqrt(np.mean(np.square(y_actual-y_predicted)))
    return RMSE

def CVRMSE(y_predicted, y_actual):
    y_predicted = np.asarray(y_predicted, dtype=float)
    y_actual = np.asarray(y_actual, dtype=float)

    RMSE = np.sqrt(np.mean(np.square(y_actual-y_predicted)))
    CVRMSE = RMSE/np.mean(y_actual)
    return CVRMSE

def MAPE(y_predicted, y_actual):
    y_predicted = np.asarray(y_predicted, dtype=float)
    y_actual = np.asarray(y_actual, dtype=float)

    MAPE=np.mean(abs((y_actual-y_predicted)/y_actual))
    return MAPE

def MSE(y_predicted, y_actual):
    y_predicted = np.asarray(y_predicted, dtype=float)
    y_actual = np.asarray(y_actual, dtype=float)

    MSE = np.mean(abs(y_actual-y_predicted)**2)
    return MSE

def kl_divergence(p, q):

    p = np.asarray(p, dtype=float)
    q = np.asarray(q, dtype=float)
    return np.sum(np.where(p != 0, p * np.log(p / q), 0))

def kl_divergence(y_predicted, y_actual):

    stacked_values = np.hstack((y_predicted, y_actual))
    stacked_low = np.percentile(stacked_values, 0.1)
    stacked_high = np.percentile(stacked_values, 99.9)
    bins = np.linspace(stacked_low, stacked_high, 100)

    distr = np.histogram(y_predicted, bins=bins)[0]
    distr = np.clip(distr / distr.sum(), 0.001, 0.999)
    true_distr = np.histogram( y_actual, bins=bins)[0]
    true_distr = np.clip(true_distr / true_distr.sum(), 0.001, 0.999)

    kl = entropy(distr, true_distr)
    return kl 

In [ ]:
import torch
from torch_geometric.data import InMemoryDataset, download_url


class Mytrain(InMemoryDataset):
    def __init__(self, root,transform=None, pre_transform=None):
        super().__init__(root, transform, pre_transform)
        self.data, self.slices = torch.load(self.processed_paths[0])
       # ,self.edge_index,self.y=x,edge_index,y
        
    
    @property
    def raw_file_names(self):
        return ['some_file_1', 'some_file_2', ...]
    
    @property
    def processed_file_names(self):
        return ['data.pt']

    def process(self):
        data_list=[]
        weighted_feats=[]
        for i in range(x_train.shape[0]):
            Edge_index = edge_index.type(torch.long)
            X =x_train[i]
            if feats_mode =='causal':
                t=torch.zeros(14,25) # 25 dimensions = 1+10+14 = node number, node embedding, features
              
                causal_weighted= np.multiply(np.array(X),np.array(ate_list))
                weighted_feats.append(causal_weighted)
                
                for j in range(14): # for j-th feature
                  t[j]=torch.from_numpy(np.concatenate(( [float(X[int(j)])], # node (1d)
                                                        list(lst_dw[int(j)]), # node embeddings (10d), prior embeddings from DeepWalk/Node2Vec
                                                        list(causal_weighted) # feature (10d), causally weighted features        
                                                        ))) 
            elif feats_mode =='equal':
                t=torch.zeros(14,25) # 25 dimensions = 1+10+14 = node number, node embedding, features
              
                weighted= np.array(X)
                weighted_feats.append(weighted)
                
                for j in range(14): # for j-th feature
                  t[j]=torch.from_numpy(np.concatenate(( [float(X[int(j)])], # node (1d)
                                                        list(lst_dw[int(j)]), # node embeddings (10d), prior embeddings from DeepWalk/Node2Vec
                                                        list(weighted) # feature (10d), equally weighted features        
                                                        ))) 
            else:
                t=torch.zeros(14,11) # 11 dimensions = 1+10 = node number, node embedding
                                    
                
                for j in range(14):
                  t[j]=torch.from_numpy(np.concatenate(( [float(X[int(j)])], # node (1d)
                                                        list(lst_dw[int(j)]), # node embeddings (10d), prior embeddings from DeepWalk/Node2Vec
                                                        ))) 
                  

            Y = y_train[i].reshape(-1,1).to(torch.float32)
            data = Data(x=t, edge_index=Edge_index, y=Y)
      
            data_list.append(data)

        if self.pre_filter is not None:
            data_list = [data for data in data_list if self.pre_filter(data)]

        if self.pre_transform is not None:
            data_list = [self.pre_transform(data) for data in data_list]
        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])


class Mytest(InMemoryDataset):
    def __init__(self, root,transform=None, pre_transform=None):
        super().__init__(root, transform, pre_transform)
        self.data, self.slices = torch.load(self.processed_paths[0])
       # ,self.edge_index,self.y=x,edge_index,y
        
    
    @property
    def raw_file_names(self):
        return ['some_file_1', 'some_file_2', ...]
    
    @property
    def processed_file_names(self):
        return ['data.pt']

    def process(self):
        data_list=[]
        weighted_feats=[]
        for i in range(x_test.shape[0]):
            Edge_index = edge_index.type(torch.long)
            X =x_test[i]

            if feats_mode =='causal':
                t=torch.zeros(14,25) # 25 dimensions = 1+10+14 = node number, node embedding, features
              
                causal_weighted= np.multiply(np.array(X),np.array(ate_list))
                weighted_feats.append(causal_weighted)
                
                for j in range(14): # for j-th feature
                  t[j]=torch.from_numpy(np.concatenate(( [float(X[int(j)])], # node (1d)
                                                        list(lst_dw[int(j)]), # node embeddings (10d), prior embeddings from DeepWalk/Node2Vec
                                                        list(causal_weighted) # feature (10d), causally weighted features        
                                                        ))) 
            elif feats_mode =='equal':
                t=torch.zeros(14,25) # 25 dimensions = 1+10+14 = node number, node embedding, features
              
                weighted= np.array(X)
                weighted_feats.append(weighted)
                
                for j in range(14): # for j-th feature
                  t[j]=torch.from_numpy(np.concatenate(( [float(X[int(j)])], # node (1d)
                                                        list(lst_dw[int(j)]), # node embeddings (10d), prior embeddings from DeepWalk/Node2Vec
                                                        list(weighted) # feature (10d), equally weighted features        
                                                        ))) 
            else:
                t=torch.zeros(14,11) # 11 dimensions = 1+10 = node number, node embedding
                                    
                
                for j in range(14):
                  t[j]=torch.from_numpy(np.concatenate(( [float(X[int(j)])], # node (1d)
                                                        list(lst_dw[int(j)]), # node embeddings (10d), prior embeddings from DeepWalk/Node2Vec
                                                        ))) 
            
            Y = y_test[i].reshape(-1,1).to(torch.float32)
            data = Data(x=t, edge_index=Edge_index, y=Y)
        
            data_list.append(data)

        if self.pre_filter is not None:
            data_list = [data for data in data_list if self.pre_filter(data)]

        if self.pre_transform is not None:
            data_list = [self.pre_transform(data) for data in data_list]
        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])

In [ ]:
feats_mode='causal'
#feats_mode ='equal' 
#feats_mode ='noweighting' 

col=df_train.columns
y_train=torch.from_numpy(np.array(df_train['y'])).reshape(df_train.shape[0],1).to(torch.float32)
x_train=torch.from_numpy(np.array(df_train[[i for i in col[:-4]]+["visit","T"]])).to(torch.float32)
Mydata_train=Mytrain(".\mydata_sampled_ate\MYdata_train")

y_test=torch.from_numpy(np.array(df_test['y'])).reshape(df_test.shape[0],1).to(torch.float32)
x_test=torch.from_numpy(np.array(df_test[[i for i in col[:-4]]+["visit","T"]])).to(torch.float32)
Mydata_test=Mytest(".\mydata_sampled_ate\MYdata_test")

Processing...
Done!
Processing...
Done!
Processing...
Done!


# GCN

In [ ]:
def train():
    model.train()
    loss_all = 0
    y_actual = []
    y_predicted = []
    loss_all=0
    for data in train_loader:
        loss = 0
        data = data.to(device)
        optimizer.zero_grad()
        output = model(data)
        label = data.y.to(device)
        loss = crit(output, label)
        loss.backward()
        loss_all += data.num_graphs * loss.item()
        optimizer.step()
        y_actual +=(label).cpu().detach().ravel().tolist()
        y_predicted +=(output).cpu().detach().ravel().tolist()
    
    
    loss=loss_all/len(Mydata_train)
    r2=R2(y_predicted, y_actual)
    mse = MSE(y_predicted, y_actual)
    kl=kl_divergence(y_predicted, y_actual)

    print("R2:%f" % (R2(y_predicted, y_actual)),end='  ')
    print("MSE:%f" % (MSE(y_predicted, y_actual)),end='  ')
    print("KL:%f" % (kl_divergence(y_predicted, y_actual)),end='  ')
    print("MAE:%f" % (MAE(y_predicted, y_actual)),end='  ')
    print("RMSE:%f" % (RMSE(y_predicted, y_actual)),end='  ')
    print("CVRMSE:%f" % (CVRMSE(y_predicted, y_actual)),end='  ')

    return loss,r2,mse,kl


def val():
    model.eval()
    y_actual = []
    y_predicted = []
    loss_all=0
    for data in test_loader:
      loss = 0
      data = data.to(device)
      output = model(data)
      label = data.y.to(device)
      y_actual +=(label).cpu().detach().ravel().tolist()
      y_predicted +=(output).cpu().detach().ravel().tolist()
      loss = crit(output, label)
      loss_all += loss.item()

    
    loss = loss_all / len(Mydata_test)
    r2=R2(y_predicted, y_actual)
    mse = MSE(y_predicted, y_actual)
    kl=kl_divergence(y_predicted, y_actual)

    print("R2:%f" % (R2(y_predicted, y_actual)),end='  ')
    print("MSE:%f" % (MSE(y_predicted, y_actual)),end='  ')
    print("KL:%f" % (kl_divergence(y_predicted, y_actual)),end='  ')
    print("MAE:%f" % (MAE(y_predicted, y_actual)),end='  ')
    print("RMSE:%f" % (RMSE(y_predicted, y_actual)),end='  ')
    print("CVRMSE:%f" % (CVRMSE(y_predicted, y_actual)),end='  ')

    return loss,r2, mse,kl

In [ ]:
import torch
from torch_scatter import scatter_add
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(25, 64)
        self.conv2 = GCNConv(64, 10)
        # self-attention layer
        
        self.f1 = torch.nn.Linear(140,32)
        self.f2 = torch.nn.Linear(32,1)
    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = self.attention
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = x.reshape(-1,140)
        x = self.f1(x)
        x = self.f2(x)
        return x

In [ ]:
num_epochs = 2560
batch_size = 512
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.002, weight_decay=5e-4)
crit = F.mse_loss
train_loader = DataLoader(Mydata_train, batch_size=batch_size)
test_loader = DataLoader(Mydata_test, batch_size=512)

for epoch in range(num_epochs):
    loss,r2,mse,kl=train()
    if epoch %5==0:      
        print('train_loss:')
        print(loss)

        loss,r2,mse,kl=val()
        print('test_loss:')
        print(loss)

y_predicted = []
for data in test_loader:
    loss = 0
    data = data.to(device)
    output = model(data)
    label = data.y.to(device)
    y_predicted +=(output).cpu().detach().ravel().tolist()
df_test["y_hat"]=y_predicted

R2:-0.024036  MSE:0.002990  KL:1.247097  MAE:0.009826  RMSE:0.054681  CVRMSE:18.672716  train_loss:
0.002989981582357814
R2:0.072559  MSE:0.002796  KL:0.121532  MAE:0.006606  RMSE:0.052879  CVRMSE:17.486027  test_loss:
5.494521631923732e-06
R2:0.087444  MSE:0.002664  KL:2.115198  MAE:0.007876  RMSE:0.051619  CVRMSE:17.627042  R2:0.100940  MSE:0.002625  KL:1.866433  MAE:0.007405  RMSE:0.051235  CVRMSE:17.496212  R2:0.103677  MSE:0.002617  KL:1.270331  MAE:0.007245  RMSE:0.051157  CVRMSE:17.469562  R2:0.104247  MSE:0.002615  KL:1.318326  MAE:0.007226  RMSE:0.051141  CVRMSE:17.464006  R2:0.103408  MSE:0.002618  KL:1.308779  MAE:0.007226  RMSE:0.051165  CVRMSE:17.472183  train_loss:
0.0026178686612746737
R2:0.118098  MSE:0.002659  KL:0.053999  MAE:0.005711  RMSE:0.051564  CVRMSE:17.051327  test_loss:
5.2227962901602e-06
R2:0.106504  MSE:0.002609  KL:1.383942  MAE:0.007235  RMSE:0.051077  CVRMSE:17.441997  R2:0.105358  MSE:0.002612  KL:1.017340  MAE:0.007164  RMSE:0.051109  CVRMSE:17.453179

R2:0.104883  MSE:0.002614  KL:0.500720  MAE:0.007134  RMSE:0.051123  CVRMSE:17.457811  R2:0.105849  MSE:0.002611  KL:0.589760  MAE:0.007085  RMSE:0.051095  CVRMSE:17.448388  R2:0.105549  MSE:0.002612  KL:0.829491  MAE:0.007133  RMSE:0.051104  CVRMSE:17.451313  R2:0.106632  MSE:0.002608  KL:0.835054  MAE:0.007149  RMSE:0.051073  CVRMSE:17.440747  R2:0.105095  MSE:0.002613  KL:0.666143  MAE:0.007129  RMSE:0.051117  CVRMSE:17.455743  train_loss:
0.0026129447496954484
R2:0.118220  MSE:0.002658  KL:0.056525  MAE:0.005706  RMSE:0.051560  CVRMSE:17.050140  test_loss:
5.221919713687387e-06
R2:0.105654  MSE:0.002611  KL:0.826030  MAE:0.007149  RMSE:0.051101  CVRMSE:17.450283  R2:0.104463  MSE:0.002615  KL:0.742459  MAE:0.007128  RMSE:0.051135  CVRMSE:17.461899  R2:0.106275  MSE:0.002609  KL:0.950716  MAE:0.007139  RMSE:0.051083  CVRMSE:17.444227  R2:0.109134  MSE:0.002601  KL:0.631706  MAE:0.007132  RMSE:0.051001  CVRMSE:17.416301  R2:0.107887  MSE:0.002605  KL:0.613254  MAE:0.007109  RMSE:0.05

R2:0.106180  MSE:0.002610  KL:0.856136  MAE:0.007103  RMSE:0.051086  CVRMSE:17.445153  R2:0.107398  MSE:0.002606  KL:0.757668  MAE:0.007099  RMSE:0.051051  CVRMSE:17.433269  R2:0.106007  MSE:0.002610  KL:0.819236  MAE:0.007143  RMSE:0.051091  CVRMSE:17.446839  R2:0.104498  MSE:0.002615  KL:0.734635  MAE:0.007113  RMSE:0.051134  CVRMSE:17.461565  R2:0.104122  MSE:0.002616  KL:0.695199  MAE:0.007152  RMSE:0.051145  CVRMSE:17.465231  train_loss:
0.002615786017936153
R2:0.116946  MSE:0.002662  KL:0.054055  MAE:0.005199  RMSE:0.051598  CVRMSE:17.062456  test_loss:
5.229684696113949e-06
R2:0.104932  MSE:0.002613  KL:0.640522  MAE:0.007125  RMSE:0.051122  CVRMSE:17.457330  R2:0.104949  MSE:0.002613  KL:0.636439  MAE:0.007101  RMSE:0.051121  CVRMSE:17.457166  R2:0.104573  MSE:0.002614  KL:0.703129  MAE:0.007103  RMSE:0.051132  CVRMSE:17.460831  R2:0.106391  MSE:0.002609  KL:0.750826  MAE:0.007116  RMSE:0.051080  CVRMSE:17.443098  R2:0.106912  MSE:0.002608  KL:0.610168  MAE:0.007062  RMSE:0.051

R2:0.106764  MSE:0.002608  KL:0.846119  MAE:0.007085  RMSE:0.051069  CVRMSE:17.439458  R2:0.107219  MSE:0.002607  KL:0.808079  MAE:0.007093  RMSE:0.051056  CVRMSE:17.435009  R2:0.106676  MSE:0.002608  KL:0.662996  MAE:0.007062  RMSE:0.051072  CVRMSE:17.440311  R2:0.104700  MSE:0.002614  KL:0.694636  MAE:0.007126  RMSE:0.051128  CVRMSE:17.459592  R2:0.107643  MSE:0.002606  KL:0.604215  MAE:0.007070  RMSE:0.051044  CVRMSE:17.430875  train_loss:
0.0026055049804096686
R2:0.116828  MSE:0.002663  KL:0.053670  MAE:0.005272  RMSE:0.051601  CVRMSE:17.063594  test_loss:
5.230510571119722e-06
R2:0.106566  MSE:0.002609  KL:0.685018  MAE:0.007103  RMSE:0.051075  CVRMSE:17.441388  R2:0.106207  MSE:0.002610  KL:1.524329  MAE:0.007126  RMSE:0.051085  CVRMSE:17.444888  R2:0.104788  MSE:0.002614  KL:0.640959  MAE:0.007101  RMSE:0.051126  CVRMSE:17.458738  R2:0.105944  MSE:0.002610  KL:0.835302  MAE:0.007112  RMSE:0.051093  CVRMSE:17.447459  R2:0.106574  MSE:0.002609  KL:0.676395  MAE:0.007100  RMSE:0.05

R2:0.104038  MSE:0.002616  KL:0.868195  MAE:0.007098  RMSE:0.051147  CVRMSE:17.466047  R2:0.106811  MSE:0.002608  KL:0.937965  MAE:0.007090  RMSE:0.051068  CVRMSE:17.438996  R2:0.106750  MSE:0.002608  KL:0.870858  MAE:0.007098  RMSE:0.051070  CVRMSE:17.439596  R2:0.105701  MSE:0.002611  KL:0.561586  MAE:0.007106  RMSE:0.051100  CVRMSE:17.449833  R2:0.105838  MSE:0.002611  KL:0.723978  MAE:0.007069  RMSE:0.051096  CVRMSE:17.448488  train_loss:
0.002610773017572402
R2:0.118499  MSE:0.002658  KL:0.052539  MAE:0.005082  RMSE:0.051552  CVRMSE:17.047445  test_loss:
5.2205035878948385e-06
R2:0.106504  MSE:0.002609  KL:0.951499  MAE:0.007085  RMSE:0.051077  CVRMSE:17.441997  R2:0.104698  MSE:0.002614  KL:0.741627  MAE:0.007110  RMSE:0.051128  CVRMSE:17.459616  R2:0.105433  MSE:0.002612  KL:0.691565  MAE:0.007083  RMSE:0.051107  CVRMSE:17.452446  R2:0.106823  MSE:0.002608  KL:0.909206  MAE:0.007063  RMSE:0.051068  CVRMSE:17.438880  R2:0.105448  MSE:0.002612  KL:0.714220  MAE:0.007123  RMSE:0.05

R2:0.107037  MSE:0.002607  KL:0.959610  MAE:0.007068  RMSE:0.051061  CVRMSE:17.436792  R2:0.104786  MSE:0.002614  KL:0.745187  MAE:0.007112  RMSE:0.051126  CVRMSE:17.458754  R2:0.106993  MSE:0.002607  KL:0.935247  MAE:0.007078  RMSE:0.051063  CVRMSE:17.437223  R2:0.104253  MSE:0.002615  KL:0.718808  MAE:0.007107  RMSE:0.051141  CVRMSE:17.463951  R2:0.104076  MSE:0.002616  KL:0.748343  MAE:0.007080  RMSE:0.051146  CVRMSE:17.465678  train_loss:
0.002615919916818568
R2:0.114113  MSE:0.002671  KL:0.052568  MAE:0.005230  RMSE:0.051680  CVRMSE:17.089801  test_loss:
5.246769149613519e-06
R2:0.104236  MSE:0.002615  KL:0.911292  MAE:0.007099  RMSE:0.051142  CVRMSE:17.464120  R2:0.104551  MSE:0.002615  KL:0.710039  MAE:0.007107  RMSE:0.051133  CVRMSE:17.461049  R2:0.104578  MSE:0.002614  KL:0.756043  MAE:0.007111  RMSE:0.051132  CVRMSE:17.460782  R2:0.106367  MSE:0.002609  KL:0.569489  MAE:0.007053  RMSE:0.051081  CVRMSE:17.443333  R2:0.104595  MSE:0.002614  KL:0.835764  MAE:0.007108  RMSE:0.051

KeyboardInterrupt: 

In [ ]:
# estimate area under the uplift curve (AUUC)
from causalml.metrics import *
uplift=df_test.copy()
tau_hat=pd.concat([t0,t1], axis=0, join="inner")
uplift=pd.concat([uplift,tau_hat], axis=1, join="inner")
uplift = uplift.loc[:,~uplift.columns.duplicated()]

auuc=auuc_score(uplift, outcome_col='y', treatment_col='T')
gcn_auuc=pd.DataFrame(auuc[["y_hat","Random"]],columns=['auuc'])
gcn_auuc

,auuc
y_hat,0.732616
Random,0.499185


# GAT

In [ ]:
from uuid import RFC_4122
import torch
import math
from torch_geometric.nn import MessagePassing
from torch_geometric.nn import GATConv

from torch_geometric.utils import add_self_loops,degree
from torch_geometric.datasets import Planetoid
import ssl
import torch.nn.functional as F


class Net(torch.nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.gat1=GATConv(in_channels=25,out_channels=8,heads=8,dropout=0.6)
        self.gat2=GATConv(in_channels=64,out_channels=10,heads=1,dropout=0.6)
        self.f1 = torch.nn.Linear(140,32)
        self.f2 = torch.nn.Linear(32,1)
    def forward(self,data):
        x,edge_index=data.x, data.edge_index
        x=self.gat1(x,edge_index)
        x=self.gat2(x,edge_index)
        x=x.reshape(-1,140)
        x = self.f1(x)
        x = self.f2(x)
        return x

ssl._create_default_https_context = ssl._create_unverified_context
def train():
    model.train()
    loss_all = 0
    y_actual = []
    y_predicted = []
    loss_all=0
    for data in train_loader:
        loss = 0
        data = data.to(device)
        optimizer.zero_grad()
        output = model(data)
        label = data.y.to(device)
        loss = crit(output, label)
        loss.backward()
        loss_all += data.num_graphs * loss.item()
        optimizer.step()
        y_actual +=(label).cpu().detach().ravel().tolist()
        y_predicted +=(output).cpu().detach().ravel().tolist()
    
    
    loss=loss_all/len(Mydata_train)
    r2=R2(y_predicted, y_actual)
    mse = MSE(y_predicted, y_actual)
    kl=kl_divergence(y_predicted, y_actual)

    print("R2:%f" % (R2(y_predicted, y_actual)),end='  ')
    print("MSE:%f" % (MSE(y_predicted, y_actual)),end='  ')
    print("KL:%f" % (kl_divergence(y_predicted, y_actual)),end='  ')
    print("MAE:%f" % (MAE(y_predicted, y_actual)),end='  ')
    print("RMSE:%f" % (RMSE(y_predicted, y_actual)),end='  ')
    print("CVRMSE:%f" % (CVRMSE(y_predicted, y_actual)),end='  ')

    return loss,r2,mse,kl


def val():
    model.eval()
    y_actual = []
    y_predicted = []
    loss_all=0
    for data in test_loader:
      loss = 0
      data = data.to(device)
      output = model(data)
      label = data.y.to(device)
      y_actual +=(label).cpu().detach().ravel().tolist()
      y_predicted +=(output).cpu().detach().ravel().tolist()
      loss = crit(output, label)
      loss_all += loss.item()

    
    loss = loss_all / len(Mydata_test)
    r2=R2(y_predicted, y_actual)
    mse = MSE(y_predicted, y_actual)
    kl=kl_divergence(y_predicted, y_actual)

    print("R2:%f" % (R2(y_predicted, y_actual)),end='  ')
    print("MSE:%f" % (MSE(y_predicted, y_actual)),end='  ')
    print("KL:%f" % (kl_divergence(y_predicted, y_actual)),end='  ')
    print("MAE:%f" % (MAE(y_predicted, y_actual)),end='  ')
    print("RMSE:%f" % (RMSE(y_predicted, y_actual)),end='  ')
    print("CVRMSE:%f" % (CVRMSE(y_predicted, y_actual)),end='  ')

    return loss,r2, mse,kl



num_epochs = 256
batch_size = 512
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.002, weight_decay=5e-4)
crit = F.mse_loss
train_loader = DataLoader(Mydata_train, batch_size=batch_size)
test_loader = DataLoader(Mydata_test, batch_size=512)

for epoch in range(num_epochs):
    loss,r2,mse,kl=train()
    if epoch %5==0:      
        print('train_loss:')
        print(loss)

        loss,r2,mse,kl=val()
        print('test_loss:')
        print(loss)

y_predicted = []
for data in test_loader:
    loss = 0
    data = data.to(device)
    output = model(data)
    label = data.y.to(device)
    y_predicted +=(output).cpu().detach().ravel().tolist()
df_test["y_hat"]=y_predicted

R2:-0.813384  MSE:0.005295  KL:2.094833  MAE:0.017525  RMSE:0.072765  CVRMSE:24.848187  train_loss:
0.005294719280111404
R2:0.072054  MSE:0.002798  KL:0.126590  MAE:0.005984  RMSE:0.052893  CVRMSE:17.490782  test_loss:
5.497692996261892e-06
R2:0.060488  MSE:0.002743  KL:2.104842  MAE:0.009919  RMSE:0.052375  CVRMSE:17.885496  R2:0.057913  MSE:0.002751  KL:1.238602  MAE:0.008663  RMSE:0.052447  CVRMSE:17.909985  R2:0.060253  MSE:0.002744  KL:1.248182  MAE:0.008433  RMSE:0.052382  CVRMSE:17.887734  R2:0.064215  MSE:0.002732  KL:1.533469  MAE:0.008456  RMSE:0.052271  CVRMSE:17.849982  R2:0.064579  MSE:0.002731  KL:1.586445  MAE:0.008461  RMSE:0.052261  CVRMSE:17.846508  train_loss:
0.0027312411740449236
R2:0.056647  MSE:0.002844  KL:0.100752  MAE:0.005517  RMSE:0.053330  CVRMSE:17.635387  test_loss:
5.589489941400884e-06
R2:0.064812  MSE:0.002731  KL:1.744580  MAE:0.008459  RMSE:0.052255  CVRMSE:17.844289  R2:0.065108  MSE:0.002730  KL:1.738378  MAE:0.008456  RMSE:0.052247  CVRMSE:17.8414

R2:0.064456  MSE:0.002732  KL:1.592869  MAE:0.008475  RMSE:0.052265  CVRMSE:17.847686  R2:0.064599  MSE:0.002731  KL:1.694082  MAE:0.008464  RMSE:0.052261  CVRMSE:17.846322  R2:0.064635  MSE:0.002731  KL:1.742830  MAE:0.008471  RMSE:0.052260  CVRMSE:17.845979  R2:0.064600  MSE:0.002731  KL:1.617236  MAE:0.008486  RMSE:0.052261  CVRMSE:17.846311  R2:0.064951  MSE:0.002730  KL:1.751280  MAE:0.008426  RMSE:0.052251  CVRMSE:17.842962  train_loss:
0.002730155742312368
R2:0.056884  MSE:0.002843  KL:0.100740  MAE:0.005550  RMSE:0.053324  CVRMSE:17.633171  test_loss:
5.588077468592933e-06
R2:0.064745  MSE:0.002731  KL:1.648253  MAE:0.008481  RMSE:0.052257  CVRMSE:17.844927  R2:0.064765  MSE:0.002731  KL:1.741393  MAE:0.008428  RMSE:0.052256  CVRMSE:17.844735  R2:0.065008  MSE:0.002730  KL:1.755607  MAE:0.008450  RMSE:0.052249  CVRMSE:17.842421  R2:0.064726  MSE:0.002731  KL:1.748324  MAE:0.008494  RMSE:0.052257  CVRMSE:17.845111  R2:0.064690  MSE:0.002731  KL:1.677063  MAE:0.008428  RMSE:0.052

R2:0.064303  MSE:0.002732  KL:1.664241  MAE:0.008439  RMSE:0.052269  CVRMSE:17.849148  R2:0.064583  MSE:0.002731  KL:1.576043  MAE:0.008445  RMSE:0.052261  CVRMSE:17.846474  R2:0.064168  MSE:0.002732  KL:1.796350  MAE:0.008418  RMSE:0.052273  CVRMSE:17.850430  R2:0.064346  MSE:0.002732  KL:1.714394  MAE:0.008441  RMSE:0.052268  CVRMSE:17.848737  R2:0.065047  MSE:0.002730  KL:1.782470  MAE:0.008482  RMSE:0.052248  CVRMSE:17.842051  train_loss:
0.002729876980947571
R2:0.058913  MSE:0.002837  KL:0.106155  MAE:0.006180  RMSE:0.053266  CVRMSE:17.614195  test_loss:
5.5759237337438345e-06
R2:0.064603  MSE:0.002731  KL:1.704743  MAE:0.008488  RMSE:0.052261  CVRMSE:17.846283  R2:0.065151  MSE:0.002730  KL:1.568244  MAE:0.008501  RMSE:0.052245  CVRMSE:17.841057  R2:0.064490  MSE:0.002732  KL:1.684507  MAE:0.008433  RMSE:0.052264  CVRMSE:17.847359  R2:0.064327  MSE:0.002732  KL:1.678769  MAE:0.008466  RMSE:0.052268  CVRMSE:17.848919  R2:0.064538  MSE:0.002731  KL:1.686070  MAE:0.008441  RMSE:0.05

R2:0.064688  MSE:0.002731  KL:1.731077  MAE:0.008439  RMSE:0.052258  CVRMSE:17.845475  R2:0.064379  MSE:0.002732  KL:1.767459  MAE:0.008429  RMSE:0.052267  CVRMSE:17.848424  R2:0.064461  MSE:0.002732  KL:1.777455  MAE:0.008439  RMSE:0.052265  CVRMSE:17.847634  R2:0.064899  MSE:0.002730  KL:1.683874  MAE:0.008465  RMSE:0.052252  CVRMSE:17.843460  R2:0.063806  MSE:0.002733  KL:1.786041  MAE:0.008441  RMSE:0.052283  CVRMSE:17.853883  train_loss:
0.002733499015558268
R2:0.056011  MSE:0.002846  KL:0.099016  MAE:0.005763  RMSE:0.053348  CVRMSE:17.641328  test_loss:
5.593251218773543e-06
R2:0.064669  MSE:0.002731  KL:1.590624  MAE:0.008481  RMSE:0.052259  CVRMSE:17.845651  R2:0.064686  MSE:0.002731  KL:1.735661  MAE:0.008474  RMSE:0.052258  CVRMSE:17.845490  R2:0.065101  MSE:0.002730  KL:1.621649  MAE:0.008488  RMSE:0.052247  CVRMSE:17.841530  R2:0.064795  MSE:0.002731  KL:1.760341  MAE:0.008435  RMSE:0.052255  CVRMSE:17.844450  R2:0.064473  MSE:0.002732  KL:1.624434  MAE:0.008470  RMSE:0.052

In [ ]:
# estimate area under the uplift curve (AUUC)
from causalml.metrics import *
uplift=df_test.copy()
tau_hat=pd.concat([t0,t1], axis=0, join="inner")
uplift=pd.concat([uplift,tau_hat], axis=1, join="inner")
uplift = uplift.loc[:,~uplift.columns.duplicated()]

auuc=auuc_score(uplift, outcome_col='y', treatment_col='T', treatment_effect_col='tau')
gat_auuc=pd.DataFrame(auuc[["y_hat","Random"]],columns=['auuc'])
gat_auuc

,auuc
y_hat,0.880700
Random,0.499185


In [ ]:
result=pd.DataFrame(columns=[['AUUC']])
result.loc['S Learner(LR)','AUUC']=0.497983
result.loc['S Learner(XGB)','AUUC']=0.875572
result.loc['S Learner(LGBM)','AUUC']=0.883033

result.loc['GCN (Struct)','AUUC']=0.501865
result.loc['GCN (Struct+Feature)','AUUC']=0.721959
result.loc['GCN (Struct+Causal Weighting)','AUUC']=gcn_auuc.loc["y_hat"].values

result.loc['GAT (Struct)','AUUC']=0.544286
result.loc['GAT (Struct+Feature)','AUUC']=0.847630
result.loc['GAT (Struct+Causal Weighting)','AUUC']=gat_auuc.loc["y_hat"].values

result

,AUUC
S Learner(LR),0.497983
S Learner(XGB),0.875572
S Learner(LGBM),0.883033
GCN (Struct),0.501865
GCN (Struct+Feature),0.721959
GCN (Struct+Causal Weighting),0.732616
GAT (Struct),0.544286
GAT (Struct+Feature),0.84763
GAT (Struct+Causal Weighting),0.8807
